Import libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegressionCV
import sklearn.metrics as metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.tree import export_graphviz
from IPython.display import Image
from IPython.display import display
%matplotlib inline

Data
1. [hw6_dataset.csv](https://drive.google.com/file/d/181asSju1ILeAEqNCLh1qgYi-CP6RFo7M/view?usp=sharing)
2. [hw6_dataset_missing.csv](https://drive.google.com/file/d/14JVTe06qCzouWSxatUFdJtOxvlfrFk7e/view?usp=sharing)

## Automated Breast Cancer Detection

In this homework, we will consider the problem of early breast cancer detection from X-ray images. Specifically, given a candidate region of interest (ROI) from an X-ray image of a patient's breast, the goal is to predict if the region corresponds to a malignant tumor (label 1) or is normal (label 0). The training and test data sets for this problem is provided in the file `hw6_dataset.csv`. Each row in these files corresponds to a ROI in a patient's X-ray, with columns 1-117 containing features computed using standard image processing algorithms. The last column contains the class label, and is based on a radiologist's opinion or a biopsy. This data was obtained from the KDD Cup 2008 challenge.

The data set contain a total of 69,098 candidate ROIs, of which only 409 are malignant, while the remaining are all normal. 

*Note*: be careful of reading/treating column names and row names in this data set.

## Question 1: Beyond Classification Accuracy


0.  Split the data set into a training set and a testing set.  The training set should be 75% of the original data set, and the testing set 25%.  Use `np.random.seed(9001)`.

1. Fit a logistic regression classifier to the training set and report the  accuracy of the classifier on the test set. You should use $L_2$ regularization in logistic regression, with the regularization parameter tuned using cross-validation. 
    1. How does the fitted model compare with a classifier that predicts 'normal' (label 0) on all patients? 
    2. Do you think the difference in the classification accuracies are large enough to declare logistic regression as a better classifier than the all 0's classifier? Why or why not?
    
For applications with imbalanced class labels, in this case when there are many more healthy subjects ($Y=0$) than those with cancer ($Y=1$), the classification accuracy may not be the best metric to evaluate a classifier's performance. As an alternative, we could analyze the confusion table for the classifier. 

<ol start="3">
<li> Compute the confusion table for both the fitted classifier and the classifier that predicts all 0's.</li>
<li> Using the entries of the confusion table compute the *true positive rate* and the *true negative rate* for the two classifiers. Explain what these evaluation metrics mean for the specific task of cancer detection. Based on the observed metrics, comment on whether the fitted model is better than the all 0's classifier.</li>
<li> What is the *false positive rate* of the fitted classifier, and how is it related to its true positive and true negative rate? Why is a classifier with high false positive rate undesirable for a cancer detection task?</li>
</ol>
*Hint:* You may use the `metrics.confusion_matrix` function to compute the confusion matrix for a classification model.

In [ ]:
df = pd.read_csv('/content/sample_data/hw6_dataset.csv', header = None)
np.random.seed(9001)
msk = np.random.rand(len(df)) < 0.75
df_train = df[msk]
df_test = df[~msk]
df_train = df_train.dropna()
df_test = df_test.dropna()
df_train.head()
print(df_train.shape)
print(df_test.shape)

(52009, 118)
(17089, 118)


In [ ]:
X_train = df_train.iloc[:, :-1]
X_test = df_test.iloc[:, :-1]
y_train = df_train.iloc[:, -1]
y_test = df_test.iloc[:, -1]

In [ ]:
logreg = LogisticRegressionCV(random_state=0).fit(X_train, y_train)
print('Train Accuracy: \n')
print(accuracy_score(y_train, logreg.predict(X_train)))
print('\nTest Accuracy:\n')
print(accuracy_score(y_test, logreg.predict(X_test)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Train Accuracy: 

0.9955969159183987

Test Accuracy:

0.995026040142782


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
def make_confusion_matrix(pred, y_value):
  false_neg, false_pos = 0,0
  true_neg, true_pos = 0, 0
  for i, __ in enumerate(pred):
    if y_value[i] == 0:
      if pred[i] == 0:
        true_neg +=1
      else:
        false_pos +=1
    else:
      if pred[i] == 1:
        true_pos +=1
      else:
        false_neg +=1
  return np.array([np.array([false_pos, true_pos]), np.array([true_neg, false_neg])])

In [ ]:
logreg_confusion_matrix = make_confusion_matrix(list(logreg.predict(X_test)), y_test.tolist())
print('Confusion matrices\n')
print()
print(logreg_confusion_matrix)

Confusion matrices


[[    6    20]
 [16984    79]]


In [ ]:
logreg_true_pos = logreg_confusion_matrix[0][1]/(logreg_confusion_matrix[0][1] + logreg_confusion_matrix[1][1])
logreg_true_neg = logreg_confusion_matrix[1][0]/(logreg_confusion_matrix[1][0] + logreg_confusion_matrix[0][0])

In [ ]:
print("logistic regression\n\ntrue positive: ", logreg_true_pos, '\ntrue negative: ',logreg_true_neg)

logistic regression

true positive:  0.20202020202020202 
true negative:  0.9996468510888759


In [ ]:
logreg_false_pos = logreg_confusion_matrix[0][0]/(logreg_confusion_matrix[1][0] + logreg_confusion_matrix[0][0])

In [ ]:
logreg_false_pos

0.0003531489111241907

## (Optional) Question 2: Missing data

In this problem you are given a different data set, `hw6_dataset_missing.csv`, that is  similar to the one you used above (same column definitions and same conditions), however this data set contains missing values. 

*Note*: be careful of reading/treating column names and row names in this data set as well, it *may* be different than the first data set.


1. Remove all observations that contain and missing values, split the dataset into a 75-25 train-test split, and fit the regularized logistic regression as in Question 1 (use `LogisticRegressionCV` again to retune).  Report the overall classification rate and TPR in the test set.
2. Restart with a fresh copy of the data in `hw6_dataset_missing.csv` and impute the missing data via mean imputation.  Split the data 75-25 and fit the regularized logistic regression model.  Report the overall classification rate and TPR in the test set.  
3. Again restart with a fresh copy of the data in `hw6_dataset_missing.csv` and impute the missing data via a model-based imputation method. Once again split the data 75-25 and fit the regularized logistic regression model.  Report the overall classification rate and TPR in the test set.  
4. Compare the results in the 3 previous parts of this problem.  Prepare a paragraph (5-6 sentences) discussing the results, the computational complexity of the methods, and conjecture and explain why you get the results that you see.

